2. Классификация тональности коротких текстов на том же корпусе, но с помощью эмбеддингов и свёрточных нейросеточек. Основной инструмент - spacy. Пусть они научаться достраивать пайплайны в этой библиотеке примерно так https://spacy.io/usage/training#config-custom

In [ ]:
#!pip install -U pip setuptools wheel

In [ ]:
#!pip install -U spacy

In [ ]:
! python -m spacy download ru_core_news_sm

In [ ]:
import spacy
from tqdm import tqdm
from spacy.tokens import DocBin

In [ ]:
import pandas as pd
import numpy as np

dtype = {'id': np.uint16, 'date': str, 'name': str, 'text': str,  'type': np.int}
positive_df = pd.read_csv('/content/positive.csv', sep=";",  names=['id', 'date', 'name', 'text', 'type'], usecols=dtype.keys(), dtype=dtype)
negative_df = pd.read_csv('/content/negative.csv', sep=";",  names=['id', 'date', 'name', 'text', 'type'], usecols=dtype.keys(), dtype=dtype)

positive_df.drop(['id', 'date', 'name'], axis=1, inplace=True)
negative_df.drop(['id', 'date', 'name'], axis=1, inplace=True)

df = pd.concat([positive_df, negative_df], ignore_index=True)
df.rename(columns={'text': 'review', 'type': 'sentiment'}, inplace=True)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.


,review,sentiment
0,"@first_timee хоть я и школота, но поверь, у на...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1


In [ ]:
my_nlp = spacy.load("ru_core_news_sm")

In [ ]:
df.loc[(df.sentiment == -1), 'sentiment'] = 'negative'
df.loc[(df.sentiment == 1), 'sentiment'] = 'positive'

df = df.sample(frac=0.27)
df.head()

,review,sentiment
116853,@OlgaKlein Какой ужас ( какими надо быть людьм...,negative
151082,@artemandrosov одиннадцатый час ведь уже :(,negative
216890,Батодалай такой такой ! Блинн.. Ну я ему не нр...,negative
74142,"Лиза сушит юбку на батарее в школе, пока идут ...",positive
209361,1с: зарплата и управление персоналомпрограмма ...,negative


In [ ]:
df['sentiment'].value_counts()

positive    31081
negative    30164
Name: sentiment, dtype: int64

In [ ]:
data = [tuple(df.iloc[i].values) for i in range(df.shape[0])]

In [ ]:
train_data= data[:50000]
valid_data= data[50000:]

In [ ]:
def make_docs(data):
  docs = []
  a = tqdm(my_nlp.pipe(data, as_tuples=True), total = len(data))
  for doc, label in a:
    if label == 'negative':
      doc.cats['positive'] = 0
      doc.cats['negative'] = 1
    else:
      doc.cats['positive'] = 1
      doc.cats['negative'] = 0

    docs.append(doc)
  return docs


In [ ]:
len(valid_data)

11245

In [ ]:
import socket
#num_texts = 100

#train_docs = make_docs(train_data[:num_texts])
train_docs = make_docs(train_data)
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("train.spacy")

100%|██████████| 50000/50000 [03:53<00:00, 214.10it/s]


In [ ]:
#valid_docs = make_docs(valid_data[:num_texts])
valid_docs = make_docs(valid_data)
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("valid.spacy")

100%|██████████| 11245/11245 [00:51<00:00, 218.45it/s]


In [ ]:
! python -m spacy init fill-config base_config.cfg config.cfg

2022-10-27 04:54:28.507910: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
! python -m spacy train config.cfg --output ./output

2022-10-27 04:54:47.595930: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-10-27 04:54:48,961] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-10-27 04:54:48,976] [INFO] Pipeline: ['textcat']
INFO:spacy:Pipeline: ['textcat']
[2022-10-27 04:54:48,980] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-10-27 04:54:48,982] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
[2022-10-27 04:55:26,268] [INFO] Initialized pipeline components: ['textcat']
INFO:spacy:Initialized pipeline components: ['textcat']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTCAT  CATS_SCORE 

In [ ]:
print(valid_data[11244])
print(valid_data[1])
print(valid_data[1145])

('@m_prezident Государство какой плохое)\nНа героине много заработать они может?)', 'positive')
('сегодня утром что-то с дашей заржались :D обожаю ее чувство юмора', 'positive')
('RT @JuliaTensli: вот сиди потом и рыдай в конце,а потом ходи,как бомжара с размазанной тушью :(', 'negative')


In [ ]:
import spacy
# load thebest model from training
nlp = spacy.load("output/model-last")
text = ""
print("type : ‘quit’ to exit")
# predict the sentiment until someone writes quit
while text != "quit":
    text = input("Please enter example input: ")
    doc = nlp(text)
    print(doc.cats)
    if doc.cats['positive'] >.5:
        print(f"the sentiment is positive")
    else:
        print(f"the sentiment is negative")

type : ‘quit’ to exit
Please enter example input: Государство какой плохое
{'positive': 2.7320893423166126e-05, 'negative': 0.9999727010726929}
the sentiment is negative
Please enter example input: сегодня утром что-то с дашей заржались :D обожаю ее чувство юмора
{'positive': 1.0, 'negative': 2.1615366441185557e-10}
the sentiment is positive
Please enter example input: вот сиди потом и рыдай в конце,а потом ходи,как бомжара с размазанной тушью :(
{'positive': 2.5174633933033874e-08, 'negative': 1.0}
the sentiment is negative
